In [1]:
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image
import skimage.transform
import tensorflow as tf
from keras.preprocessing.image import img_to_array, load_img


In [8]:
class ImageHelper(object):
    
    def save_image(self, plot_images, epoch):
        #self to initialise, plot_images is part of a later function
        #calling a concat of real x, fake y, real y and fake x
        #epoch is the number of epochs it is
        os.makedirs('gan_images', exist_ok=True)
        #will make a directory recursively
        titles = ['Original', 'Transformed']
        fig,axs = plt.subplots(2,2,figsize=(14,16))
        cnt = 0
        for i in range(2):
            for j in range(2): 
                axs[i,j].imshow(plot_images[cnt])
                axs[i,j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig('gan_images/{}'.format(epoch))
        plt.close()
        #creating a plotting of a figure per epoch
        #opening a figure to show it and then closing the figure or the function?

    def plot20(self, images_paths_array):
        plt.figure(figsize=(10,8))
        for i in range(20):
            img = plt.imread(images_paths_array[i])
            plt.subplot(4,5, i+1)
            plt.imshow(img)
            plt.title(img.shape)
            plt.xticks([])
            plt.yticks([])
        plt.tight_layout()
        plt.show
        #plotting 20 images 
    
        #loads the image in the memory from the predefined location 
    def load_image(self,path):
        return (img_to_array(load_img(path)))
    
    def load_testing_image(self, path):
        self.img_res = (128,128)
        path_X=glob(path + '/testA/*.jpeg')
        path_Y=glob(path + '/testB/*.jpeg')
        
        image_X = np.random.choice(path_X, 1)
        image_Y = np.random.choice(path_Y, 1)
                                   
        img_X= self.load_image(image_X[0])
        img_X = skimage.transform.resize(img_X, self.img_res)
        if np.random.random() > 0.5:
            img_X = np.fliplr(img_X)
        img_X = np.array(img_X)/127.5 -1.
        #part of normalization to -1:1 
        #dealing with grayscaling sometimes 
        img_X = np.expand_dims(img_X, axis=0)
        #Insert a new axis that will appear at the axis position in the expanded array shape.
        img_Y = self.load_image(image_Y[0])
        img_Y = skimage.transform.resize(img_Y, self.img_res)
        if np.random.random() > 0.5:
            img_X = np.fliplr(img_X)
        img_Y = np.array(img_Y)/127.5 - 1.
        img_Y = np.expand_dims(img_Y, axis=0)
        return (img_X, img_Y)
        #this is calling the image and normalising it?
        
    def load_batch_of_train_images(self, path, batch_size=1):
        self.img_res = (128,128)
        path_X= glob(path + '/trainA/*.jpeg')
        path_Y = glob(path + '/trainB/*.jpeg')
        
        self.n_batches = int(min(len(path_X), len(path_Y)) / batch_size)
        total_samples = self.n_batches * batch_size
        
        path_X = np.random.choice(path_X, total_samples, replace=False)
        path_Y = np.random.choice(path_Y, total_samples, replace=False)
        
        for i in range(self.n_batches): #-1
            batch_A = path_X[i*batch_size:(i+1)*batch_size]
            batch_B = path_Y[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.load_image(img_A)
                img_B = self.load_image(img_B)
                img_A = skimage.transform.resize(np.array(img_A), self.img_res)
                img_B = skimage.transform.resize(np.array(img_B), self.img_res)
                print((img_A).shape)
                imgs_A.append(img_A)
                imgs_B.append(img_B)
            
            imgs_A = np.array(imgs_A)/127.5-1.
            imgs_B = np.array(imgs_B)/127.5-1.
            #print(imgs_A[0].shape)
            
            yield imgs_A, imgs_B
            #yield rather than return means that I am returning a generator
            #this whole function is loading a 
        
        

In [9]:
from __future__ import print_function, division 

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

from keras.layers import Input, LeakyReLU, UpSampling2D, Conv2D, Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam


In [10]:
class CycleGAN():
    def __init__(self, image_shape, cycle_lambda, image_helper):
        self.optimizer = Adam(0.0002, 0.5)
        
        self.cycle_lambda = cycle_lambda 
        self.id_lambda = 0.1 * self.cycle_lambda
        self._image_helper = image_helper
        #calling the class image helper but assigning it to a name
        self.img_shape = image_shape
        #image shape is found within the build and compile gan
    
        
        #calculating the output shape 
        patch = int(self.img_shape[0]/ 2**4)
        self.disc_patch = (patch, patch, 1)
        
        print('Build Discriminators...')
        self._discriminatorX = self._build_discriminator_model()
        self._compile_discriminator_model(self._discriminatorX)
        self._discriminatorY = self._build_discriminator_model()
        self._compile_discriminator_model(self._discriminatorY)
        
        print('Build Generators...')
        self._generatorXY = self._build_generator_model()
        self._generatorYX = self._build_generator_model()        
        
        print('Build GAN....')
        self._build_and_compile_gan()
        #what's done above is building the discriminator model and compiling the discriminator model 
        #the constructor class field, lambda, image_helper, and optimizer are initialised 
        
    def train(self, epochs, batch_size, train_data_path): 
        
        real = np.ones((batch_size,)+ self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        history = []
        
        for epoch in range(epochs):
            for i, (imagesX, imagesY) in enumerate(self._image_helper.load_batch_of_train_images(train_data_path,batch_size)): #batch_size
                print("---------------------------------------------------------")
                print("******************Epoch {} | Batch {}***************************".format(epoch, i))
                print(imagesX.shape)
                print("Generate images...")
                fakeY = self._generatorXY.predict(imagesX) ###ERROR AREA ###
                fakeX = self._generatorYX.predict(imagesY)
            
                print('Train Discriminators...')
                discriminatorX_loss_real = self._discriminatorX.train_on_batch(imagesX, real)
                discriminatorX_loss_fake = self._discriminatorX.train_on_batch(imagesX, real)
                discriminatorX_loss = 0.5 *np.add(discriminatorX_loss_real, discriminatorX_loss_fake)
             
                discriminatorY_loss_real = self._discriminatorY.train_on_batch(imagesY,real)
                discriminatorY_loss_fake = self._discriminatorY.train_on_batch(fakeY,fake)
                discriminatorY_loss = 0.5 * np.add(discriminatorY_loss_real, discriminatorY_loss_fake)
            
                mean_discriminator_loss = 0.5 *np.add(discriminatorX_loss, discriminatorY_loss)
                                                   
        
                print('Train Generators...')
                generator_loss = self.gan.train_on_batch([imagesX, imagesY],
                                                     [real,real,
                                                     imagesX, imagesY,
                                                     imagesX, imagesY])
                print('Discriminator loss: {}'.format(mean_discriminator_loss[0]))
                print('Generator loss: {}'.format(generator_loss[0]))
                print('----------------------------------------------------------')                                       
             
                                                   
                history.append({'D':mean_discriminator_loss[0], 'G':generator_loss})   
            
                                                   
                if i%100==0:
                    self._save_images('{}_{}'.format(epoch,1), train_data_path)

    def _encode__layer(self,input_layer, filters):
        layer = Conv2D(filters, kernel_size=4, strides=2, padding='same')(input_layer)                            
        layer = LeakyReLU(alpha=0.2)(layer)
        layer = InstanceNormalization()(layer)
        return layer
    
                                      
    def _decode_transform_layer(self, input_layer, forward_layer, filters):
        layer = UpSampling2D(size=2)(input_layer)
        layer = Conv2D(filters, kernel_size=4, strides=1, padding='same', activation='relu')(layer)            
        layer = InstanceNormalization()(layer)                              
        layer = Concatenate()([layer, forward_layer])
        return layer
    
#####################
## BUILD GENERATOR ##
#####################   

    def _build_generator_model(self):
        generator_input = Input(shape=self.img_shape)
                                      
        print('Build encoder....')
        encode_layer_1 = self._encode__layer(generator_input, 32);
        encode_layer_2 = self._encode__layer(encode_layer_1, 64);
        encode_layer_3 = self._encode__layer(encode_layer_2, 128);
        encode_layer_4 = self._encode__layer(encode_layer_3, 256);
                            
        print('Build Transformer - Decoder...')
        decode_transform_layer1 = self._decode_transform_layer(encode_layer_4, encode_layer_3, 128);
        decode_transform_layer2 = self._decode_transform_layer(decode_transform_layer1, encode_layer_2, 64);
        decode_transform_layer3 = self._decode_transform_layer(decode_transform_layer2, encode_layer_1, 32);
                                
                                      
        generator_model = UpSampling2D(size = 2)(decode_transform_layer3)
        generator_model = Conv2D(self.img_shape[2], kernel_size=4, strides=1, padding='same', activation='tanh')(generator_model)
         
                                      
        final_generator_model = Model(generator_input, generator_model)
        final_generator_model.summary()
        return final_generator_model
        
                                      
    def _build_discriminator_model(self):
        discriminator_input = Input(shape = self.img_shape)
        discriminator_model = Conv2D(64, kernel_size=4, strides=2, padding='same')(discriminator_input)
        discriminator_model = LeakyReLU(alpha=0.2)(discriminator_model)    
        discriminator_model = Conv2D(128, kernel_size=4, strides=2, padding='same')(discriminator_model)
        discriminator_model = LeakyReLU(alpha=0.2)(discriminator_model)  
        discriminator_model = InstanceNormalization()(discriminator_model)                              
        discriminator_model = Conv2D(256, kernel_size=4, strides=2, padding='same')(discriminator_model)                              
        discriminator_model = LeakyReLU(alpha=0.2)(discriminator_model)
        discriminator_model = InstanceNormalization()(discriminator_model)                              
        discriminator_model = Conv2D(512, kernel_size=4, strides=2, padding='same')(discriminator_model)                              
        discriminator_model = LeakyReLU(alpha = 0.2)(discriminator_model)
        discriminator_model = InstanceNormalization()(discriminator_model)
        discriminator_model = Conv2D(1, kernel_size=4, strides=1, padding='same')(discriminator_model)
                                      
        return Model(discriminator_input, discriminator_model)
                                      
    def _compile_discriminator_model(self, model):
        model.compile(loss='binary_crossentropy',
            optimizer=self.optimizer,
            metrics=['accuracy'])
        model.summary()
                                      
    def _build_and_compile_gan(self):
        
        imageX = Input(shape= self.img_shape)
        imageY = Input(shape= self.img_shape)
        
                                                    
        fakeY = self._generatorXY(imageX)
        fakeX = self._generatorYX(imageY)                              
                                      
        reconstructedX = self._generatorYX(fakeY)                              
        reconstructedY = self._generatorXY(fakeX)
                    
                                      
        imageX_id = self._generatorYX(imageX)              
        imageY_id = self._generatorXY(imageY)                
                                      
                                      
        self._discriminatorX.trainable = False                             
        self._discriminatorY.trainable = False                              
                                      
                                      
        validX = self._discriminatorX(fakeX)                              
        validY = self._discriminatorY(fakeY)                              
                                      
                                      
        self.gan = Model(inputs= [imageX, imageY],
                         outputs= [validX, validY,
                                  reconstructedX, reconstructedY,
                                  imageX_id, imageY_id])
        self.gan.compile(loss=['mse', 'mse',
                               'mae', 'mae',
                               'mae', 'mae'],
                              loss_weights=[1,1,
                                           self.cycle_lambda, self.cycle_lambda,
                                           self.id_lambda, self.id_lambda],
                              optimizer= self.optimizer)
                                    
        self.gan.summary()
                                      
    def _save_images(self, epoch, path):
        (img_X, img_Y) = self._image_helper.load_testing_image(path)
                    
        fake_Y = self._generatorXY.predict(img_X)
        fake_X = self._generatorYX.predict(img_Y)
                                      
        plot_images = np.concatenate([img_X, fake_Y, img_Y, fake_X])
                                      
        #rescaling
        plot_images = 0.5 * plot_images + 0.5
        self._image_helper.save_image(plot_images, epoch)

                                      
                                      
                                      

In [16]:

image_helper = ImageHelper()

cycle_gan_model = CycleGAN((128, 128, 3), 10.0, image_helper)

Build Discriminators...
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_78 (Conv2D)          (None, 64, 64, 64)        3136      
                                                                 
 leaky_re_lu_48 (LeakyReLU)  (None, 64, 64, 64)        0         
                                                                 
 conv2d_79 (Conv2D)          (None, 32, 32, 128)       131200    
                                                                 
 leaky_re_lu_49 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 instance_normalization_60 (  (None, 32, 32, 128)      2         
 InstanceNormalization)                                          
                                  

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_96 (Conv2D)             (None, 64, 64, 32)   1568        ['input_22[0][0]']               
                                                                                                  
 leaky_re_lu_60 (LeakyReLU)     (None, 64, 64, 32)   0           ['conv2d_96[0][0]']              
                                                                                                  
 instance_normalization_73 (Ins  (None, 64, 64, 32)  2           ['leaky_re_lu_60[0][0]']  

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_24 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_23 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_18 (Functional)          (None, 128, 128, 3)  1545425     ['input_24[0][0]',               
                                                                  'model_17[0][0]',        

In [21]:
cycle_gan_model.train(30, 10, 'cross_stitch')

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 0 | Batch 0***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.5970667898654938
Generator loss: 98.36558532714844
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 0 | Batch 1***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8869003355503082
Generator loss: 78.8717269897461
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 

Train Discriminators...
Train Generators...
Discriminator loss: 3.448673725128174
Generator loss: 125.07120513916016
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 0 | Batch 18***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.6410032212734222
Generator loss: 104.85981750488281
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 0 | Batch 19***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discr

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 1 | Batch 7***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 213.5540008544922
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 1 | Batch 8***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 219.60205078125
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3

(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 1 | Batch 24***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 162.86058044433594
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 1 | Batch 25***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 153.23602294921875
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
--------------------

Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 158.40664672851562
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 2 | Batch 14***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8511695861816406
Generator loss: 165.1302032470703
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 2 | Batch 15***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discr

...

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 27 | Batch 7***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 116.55836486816406
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 27 | Batch 8***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 97.55772399902344
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 1

(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 27 | Batch 24***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 142.9100341796875
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 27 | Batch 25***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 157.5959930419922
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
--------------------

Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 92.5291976928711
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 28 | Batch 14***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 96.15559387207031
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 28 | Batch 15***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discr

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 29 | Batch 3***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 89.46055603027344
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 29 | Batch 4***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 93.80532836914062
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 12

(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 29 | Batch 20***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 86.45907592773438
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
---------------------------------------------------------
******************Epoch 29 | Batch 21***************************
(10, 128, 128, 3)
Generate images...
Train Discriminators...
Train Generators...
Discriminator loss: 3.8562369346618652
Generator loss: 90.66107940673828
----------------------------------------------------------
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
--------------------

In [31]:
from keras.models import load_model


In [37]:
cycle_gan_model.gan.save('/Users/ameliahagiepetros/Documents/concordia-bootcamps/final_project-1')

2021-12-12 13:36:25.195201: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /Users/ameliahagiepetros/Documents/concordia-bootcamps/final_project-1/assets


In [ ]:
# print("Ploting the images…")
# filenames = np.array(glob('training_dataset/testA/*.png'))
# image_helper.plot20(filenames)